# Linear evaluation protocol

Linear evaluation: построение простых моделей на готовых эмбеддингах из SSL-UNet-masking-reconstruction на маленькой выборки в 50к эпох.

На дообученной модели в 200 эпох.

## Собираем новые эмбеддинги

In [1]:
from models.unet_ssl import UNet1D_Light, UNet1DEncoder
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1. Создаём U-Net
unet = UNet1D_Light(n_channels=14, n_classes=14, base_ch=32)
state_path = r'C:\Users\Таисия\Desktop\МФТИ\Диплом_BCI\SSL_maskingreconstruction_50k\unet_ssl_maskrecon_50k_200epoch.pth'
state = torch.load(state_path, map_location="cpu")
unet.load_state_dict(state)
unet.to(device)
unet.eval()

# 2. Создаём encoder
encoder = UNet1DEncoder(unet)
encoder.to(device)
encoder.eval()


UNet1DEncoder(
  (inc): DoubleConv1D(
    (block): Sequential(
      (0): Conv1d(14, 32, kernel_size=(3,), stride=(1,), padding=(1,))
      (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv1d(32, 32, kernel_size=(3,), stride=(1,), padding=(1,))
      (4): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (down1): Down1D(
    (block): Sequential(
      (0): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): DoubleConv1D(
        (block): Sequential(
          (0): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(1,))
          (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
          (4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_runni

Пересчитываем эмбеддинги

In [2]:
from torch.utils.data import Dataset, DataLoader
import numpy as np

class SSLDataset(Dataset):
    def __init__(self, X):
        self.X = X  # массив (N, 14, 208), float32

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return torch.tensor(self.X[idx], dtype=torch.float32)

# Загружаем готовые нормализованные эпохи
X_ready_path = r'C:\Users\Таисия\Desktop\МФТИ\Диплом_BCI\Выборки\SSL\X_ssl_small_norm.npy'
X_ssl = np.load(X_ready_path) 

dataset = SSLDataset(X_ssl)
loader = DataLoader(dataset, batch_size=128, shuffle=False)


In [4]:
all_embs = []

with torch.no_grad():
    for batch in loader:
        batch = batch.to("cpu")             # (B, 14, 208)

        feats = encoder(batch)               # (B, ch3, L3)
        emb = feats.mean(dim=-1)             # усреднение по времени → (B, ch3)

        all_embs.append(emb.cpu().numpy())


In [5]:
# Объединяем в один массив
X_emb = np.concatenate(all_embs, axis=0)   # (N, ch3)

# Проверяем
print("New embeddings shape:", X_emb.shape)
print("min/max:", X_emb.min(), X_emb.max())
print("std:", X_emb.std())


New embeddings shape: (50000, 512)
min/max: 0.0 5.883624
std: 0.16976981


In [6]:
# Сохраняем эмбеддинги
save_path = r"C:\Users\Таисия\Desktop\МФТИ\Диплом_BCI\SSL_maskingreconstruction_50k\ssl_200epoch_embeddings_mean_50k.npz"
np.savez(save_path, X=X_emb)


## Загрузка эмбеддингов и меток

In [10]:

import numpy as np
import os
import pandas as pd

# Загрузка эмбеддингов
emb_path = r"C:\Users\Таисия\Desktop\МФТИ\Диплом_BCI\SSL_maskingreconstruction_50k\ssl_200epoch_embeddings_mean_50k.npz"
npz = np.load(emb_path)
print("Ключи в npz:", npz.files)

Ключи в npz: ['X']


In [11]:
X = npz["X"]

In [12]:
# Загрузка меток

labels_path = r'C:\Users\Таисия\Desktop\МФТИ\Диплом_BCI\Выборки\SSL\y_ssl_small.npy'
y = np.load(labels_path)

print("y shape:", y.shape)
print("Уникальные метки и их распределение:", np.unique(y, return_counts=True))

# Проверяем соответствие размеров
assert X.shape[0] == y.shape[0], \
    f"Несовпадение размерностей: X={X.shape[0]}, y={y.shape[0]}"


y shape: (50000,)
Уникальные метки и их распределение: (array([0, 1]), array([44516,  5484]))


## Train / Val / Test split

- 70% train
- 15% val
- 15% test

In [13]:
from sklearn.model_selection import train_test_split

# train + temp (val+test)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, 
    test_size=0.3,          # 30% оставим под val+test
    random_state=42,
    stratify=y
)

# temp делим пополам на val и test
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.5,          # итого: 70% train, 15% val, 15% test
    random_state=42,
    stratify=y_temp
)

print("Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)


Train: (35000, 512) Val: (7500, 512) Test: (7500, 512)


## Обучение простых моделей без подбора гиперпараметров

### Общая функция для обучения и оценки модели

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report

def train_and_eval(model, model_name, 
                   X_train, y_train, 
                   X_test, y_test):
    """
    Базовая linear evaluation без использования валид. выборки.
    Обучаемся на train, оцениваемся на test.
    """
    pipe = Pipeline([
        ("scaler", StandardScaler()),
        ("clf", model),
    ])

    pipe.fit(X_train, y_train)

    y_pred = pipe.predict(X_test)
    
    # для ROC-AUC пробуем достать вероятности или decision_function
    if hasattr(pipe.named_steps['clf'], "predict_proba"):
        y_proba = pipe.predict_proba(X_test)[:, 1]
    elif hasattr(pipe.named_steps['clf'], "decision_function"):
        y_proba = pipe.decision_function(X_test)
    else:
        y_proba = None
    
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    if y_proba is not None:
        roc_auc = roc_auc_score(y_test, y_proba)
    else:
        roc_auc = None
    
    print(f"=== {model_name} ===")
    print("Accuracy:", acc)
    print("F1:", f1)
    if roc_auc is not None:
        print("ROC-AUC:", roc_auc)
    else:
        print("ROC-AUC: не посчитать (нет вероятностей/score)")
    print("\nClassification report:\n", classification_report(y_test, y_pred))
    
    return {
        "model": model_name,
        "accuracy": acc,
        "f1": f1,
        "roc_auc": roc_auc,
    }


## LogisticRegression, Random Forest и Gradient Boosting

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

results = []

# 1) Линейная модель — Logistic Regression
logreg = LogisticRegression(
    max_iter=500,
    class_weight='balanced',   # на случай дисбаланса классов
    n_jobs=-1
)
results.append(
    train_and_eval(
        logreg, "LogisticRegression",
        X_train, y_train,
        X_test, y_test
    )
)

# 2) Random Forest
rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    n_jobs=-1,
    class_weight='balanced'    # если сильный дисбаланс
)
results.append(
    train_and_eval(
        rf, "RandomForest",
        X_train, y_train,
        X_test, y_test
    )
)

# 3) Gradient Boosting (сквозь sklearn)
gb = GradientBoostingClassifier(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=3
)
results.append(
    train_and_eval(
        gb, "GradientBoosting",
        X_train, y_train,
        X_test, y_test
    )
)

# Сводная табличка с метриками
df_results = pd.DataFrame(results)
df_results


=== LogisticRegression ===
Accuracy: 0.708
F1: 0.32573891625615764
ROC-AUC: 0.7431496018811815

Classification report:
               precision    recall  f1-score   support

           0       0.94      0.72      0.81      6678
           1       0.22      0.64      0.33       822

    accuracy                           0.71      7500
   macro avg       0.58      0.68      0.57      7500
weighted avg       0.86      0.71      0.76      7500

=== RandomForest ===
Accuracy: 0.8928
F1: 0.05188679245283019
ROC-AUC: 0.6746827837930992

Classification report:
               precision    recall  f1-score   support

           0       0.89      1.00      0.94      6678
           1       0.85      0.03      0.05       822

    accuracy                           0.89      7500
   macro avg       0.87      0.51      0.50      7500
weighted avg       0.89      0.89      0.85      7500



KeyboardInterrupt: 